In [ ]:
#워터셰드
"""
워터셰드: 강물이 한 줄기로 흐르다가 갈라지는 경계
경계를 찾는 방법 중 하나로 픽셀 값의 크기를 산과 골짜기 같은 높고 낮은 지형으로 보고
물을 채워서 그 물이 만나는 곳을 경계로 찾는 방식
floodfill과 비슷한 방식으로 연속된 영역을 찾는것인데
이때 처음 찾을 지점인 seed를 하나가 아닌 여러 곳을 사용하고 이것을 마커라 함.

cv2.watershed(img,markers)
  - img:입력영상
  - markers:마커, 입력 영상과 크기가 같은 1차원 배열(int 32)
  
markers는 입력 영상의 행과 열 크기가 같은 1차원 배열로 dtype=np.int32로 생성해야 함.
markers의 값은 경계를 찾고자 하는 픽셀 영역은 0을 갖게 하고 
연결이 된 영역이 확실한 픽셀에 대해서는 동일한 양의 정수를 값으로 갖게 함.

이 함수는 markers에 0이 아닌 값들을 이용해서 같은 영역 모두를 같은 값으로 채우고 그 
경계는 -1로 채워서 반환. 반환된 마커를 이용해서 원본 영상의 연결된
나머지 영역과 경계를 찾을 수 있음.
"""

In [17]:
#마우스와 워터셰드로 배경 분리
import cv2
import numpy as np

img =cv2.imread('./img/taekwonv1.jpg')
rows, cols = img.shape[:2]
img_draw = img.copy()

#마커생성, 모든 요소는 0으로 초기화
marker = np.zeros((rows, cols), np.int32)
markerId = 1 #마커 아이디는 1에서 시작
colors = [] #마커를 선택한 픽셀의 색상값을 저장할 공간
isDragging = False  #드래그 여부 확인 변수

#마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global img_draw, marker, markerId, isDragging
    if event == cv2.EVENT_LBUTTONDOWN: #왼쪽 마우스 버튼 다운, 드래그 시작
        isDragging = True
        #각 마커의 아이디와 현 위치의 색상 값을 쌍으로 매핑해서 저장
        colors.append((markerId, img[y,x]))
    elif event == cv2.EVENT_MOUSEMOVE: #마우스 움직임
        if isDragging: #드래그 진행중
            #마우스 좌표에 해당하는 마커의 좌표에 동일한 마커 아이디로 채워 넣기
            marker[y,x] = markerId
            #마커를 표시한 곳을 빨간색 점으로 표시해서 출력
            cv2.circle(img_draw, (x,y), 3, (0,0,255), -1)
            cv2.imshow('watershed', img_draw)
    elif event == cv2.EVENT_LBUTTONUP: #왼쪽마우스 버튼 업
        if isDragging:   
            isDragging = False #드래그 중지
            #다음 마커 선택을 위해 마커 아이디 증가
            markerId +=1
    elif event == cv2.EVENT_RBUTTONDOWN: #오른쪽 마우스 버튼 다운
        #모아 놓은 마커를 이용해서 워터셰드 적용
        cv2.watershed(img, marker)
        #마커에 -1로 표시된 경계를 초록색으로 표시
        img_draw[marker == -1] = (0,255,0)
        for mid, color in colors: #선택한 마커 아이디 개수만큼 반복
            #같은 마커 아이디 값을 갖는 영역을 마커를 선택한 색상으로 채우기
            img_draw[marker==mid] = color
        cv2.imshow('watershed', img_draw) #표시한 결과 출력
        
#화면 출력
cv2.imshow('watershed', img)
cv2.setMouseCallback('watershed', onMouse)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#그랩컷

"""
그래프 컷을 기반으로 하는 알고리즘을 확장한 것으로 
사용자가 전경으로 분리할 대상 객체가 있는 사각형 좌표를 주면 대상 객체와 배경의
색상 분포를 추정해서 동일한 레이블을 가진 연결된 영역에서 배경과 전경을 분리

cv2.grabCut(img, mask, rect, bgdModel, fgdModel, itercount[, mode])
  -img:입력영상
  -mask:입력영상과 크기가 같은 1채널 배열, 배경과 전경을 구분하는 값 저장
    -cv2.GC_BGD:확실한 배경(0)
    -cv2.GC_FGD:확실한 배경(1)
    -cv2.GC_PR_BGD:아마도 배경(2)
    -cv2.GC_PR_FGD:아마도 배경(3)
  -rect:전경이 있을 것으로 추측되는 영역의 사각형 좌표 ,튜플(x1,x2,y1,y2)
  -bgdModel, fgdModel:함수 내에서 사용할 임시 배열 버퍼(재사용할 경우 수정하지 말것)
  -iterCount:반복 횟수
  -mode:동작방법
    -cv2.GC_INIT_WITH_RECT:rect에 지정한 좌표를 기준으로 그랩컷 수행
    -cv2.GC_INIT_WITH_MASK:mask에 지정한 좌표를 기준으로 그랩컷 수행
    -cv2.GC_EVAL:재시도
    
사용하는 방법은 크게 두가지
~
"""

In [37]:
#마우스와 그랩컷으로 배경 분리
import cv2
import numpy as np

img = cv2.imread('./img/taekwonv1.jpg')
img_draw = img.copy()
mask = np.zeros(img.shape[:2], dtype=np.uint8)  # 마스크 생성
rect = [0,0,0,0] #그랩컷 초기 모드
#배경 및 전경 모델 버퍼
bgdmodel = np.zeros((1,65), np.float64)
fgdmodel = np.zeros((1,65), np.float64)

#마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    if event == cv2.EVENT_LBUTTONDOWN: #왼쪽 마우스 누름
        if flags <= 2: #아무키도 안눌렀으면
            mode =cv2.GC_INIT_WITH_RECT #드래그 시작, 사각형 모드
            rect[:2] = x,y #시작 좌표 저장
    #마우스가 움직이고 왼쪽 버튼이 눌린 상태
    if event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON:
        if mode == cv2.GC_INIT_WITH_RECT: #드래그 진행중
            img_temp = img.copy()
            #드래그 사각형 화면에 표시
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x,y), (0,255,0), 2)
            cv2.imshow('img', img_temp)
        elif flags > 1: #키가 눌러진 상태
            mode = cv2.GC_INIT_WITH_MASK #마스크 모드
            if flags & cv2.EVENT_FLAG_CTRLKEY: #컨트롤 키, 분명한 전경
                #흰색 점 화면에 표시
                cv2.circle(img_draw, (x,y), 3, (255,255,255), -1)
                #마스크에 GC_FGC로 채우기
                cv2.circle(mask,(x,y), 3, cv2.GC_FGD, -1)
            if flags & cv2.EVENT_FLAG_SHIFTKEY: #시프트 키, 분명한 배경
                #검은색 점 화면에 표시
                cv2.circle(img_draw, (x,y), 3, (0,0,0), -1)
                #마스크에 GC_BGD로 채우기
                cv2.circle(mask, (x,y), 3, cv2.GC_BGD,-1)
            cv2.imshow('img',img_draw)#그려진 모습을 화면에 출력
            
        elif event == cv2.EVENT_LBUTTONUP:#마우스 왼쪽 버튼을 뗀 상태
            if mode == cv2.GC_INIT_WITH_RECT:#사각형 그리기 종료
                rect[2:] = x,y #사각형 마지막 좌표 수집
                #사각형을 그려서 화면에 출력
                cv2.rectangle(img_draw, (rect[0], rect[1]), (x,y), (255,0,0), 2)
                cv2.imshow('img', img_draw)
            #그랩컷 적용
            cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
            img2= img.copy()
            #마스크에 확실한 배경, 아마도 배경으로 표시된 영역을 0으로 채우기
            img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_FGD)] =0
            cv2.imshow('grabcut', img2) #최종결과 출력
            mode = cv2.GC_EVAL #그랩컷 모드 리셋
        #초기 화면 출력 및 마우스 이벤트 등록
        cv2.imshow('img', img)
        cv2.setMouseCallback('img', onMouse)
        while True:
            if cv2.waitKey(0) & 0xFF == 27: #esc
                break
        cv2.destroyAllWindows()
            

            
#?????      

In [34]:
import cv2
import numpy as np

img = cv2.imread('./img/taekwonv1.jpg')
img_draw = img.copy()
mask = np.zeros(img.shape[:2], dtype=np.uint8)  # 마스크 생성
rect = [0,0,0,0]    # 사각형 영역 좌표 초기화
mode = cv2.GC_EVAL  # 그랩컷 초기 모드
# 배경 및 전경 모델 버퍼
bgdmodel = np.zeros((1,65),np.float64)
fgdmodel = np.zeros((1,65),np.float64)

# 마우스 이벤트 처리 함수
def onMouse(event, x, y, flags, param):
    global mouse_mode, rect, mask, mode
    if event == cv2.EVENT_LBUTTONDOWN : # 왼쪽 마우스 누름
        if flags <= 1: # 아무 키도 안 눌렀으면
            mode = cv2.GC_INIT_WITH_RECT # 드래그 시작, 사각형 모드 ---①
            rect[:2] = x, y # 시작 좌표 저장
    # 마우스가 움직이고 왼쪽 버튼이 눌러진 상태
    elif event == cv2.EVENT_MOUSEMOVE and flags & cv2.EVENT_FLAG_LBUTTON :
        if mode == cv2.GC_INIT_WITH_RECT: # 드래그 진행 중 ---②
            img_temp = img.copy()
            # 드래그 사각형 화면에 표시
            cv2.rectangle(img_temp, (rect[0], rect[1]), (x, y), (0,255,0), 2)
            cv2.imshow('img', img_temp)
        elif flags > 1: # 키가 눌러진 상태
            mode = cv2.GC_INIT_WITH_MASK    # 마스크 모드 ---③
            if flags & cv2.EVENT_FLAG_CTRLKEY :# 컨트롤 키, 분명한 전경
                # 흰색 점 화면에 표시
                cv2.circle(img_draw,(x,y),3, (255,255,255),-1)
                # 마스크에 GC_FGD로 채우기      ---④
                cv2.circle(mask,(x,y),3, cv2.GC_FGD,-1)
            if flags & cv2.EVENT_FLAG_SHIFTKEY : # 쉬프트키, 분명한 배경
                # 검정색 점 화면에 표시
                cv2.circle(img_draw,(x,y),3, (0,0,0),-1)
                # 마스크에 GC_BGD로 채우기      ---⑤
                cv2.circle(mask,(x,y),3, cv2.GC_BGD,-1)
            cv2.imshow('img', img_draw) # 그려진 모습 화면에 출력
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 뗀 상태 ---⑥
        if mode == cv2.GC_INIT_WITH_RECT : # 사각형 그리기 종료
            rect[2:] =x, y # 사각형 마지막 좌표 수집
            # 사각형 그려서 화면에 출력 ---⑦
            cv2.rectangle(img_draw, (rect[0], rect[1]), (x, y), (255,0,0), 2)
            cv2.imshow('img', img_draw)
        # 그랩컷 적용 ---⑧
        cv2.grabCut(img, mask, tuple(rect), bgdmodel, fgdmodel, 1, mode)
        img2 = img.copy()
        # 마스크에 확실한 배경, 아마도 배경으로 표시된 영역을 0으로 채우기
        img2[(mask==cv2.GC_BGD) | (mask==cv2.GC_PR_BGD)] = 0
        cv2.imshow('grabcut', img2) # 최종 결과 출력
        mode = cv2.GC_EVAL # 그랩컷 모드 리셋
# 초기 화면 출력 및 마우스 이벤트 등록
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)
while True:    
    if cv2.waitKey(0) & 0xFF == 27 : # esc
        break
cv2.destroyAllWindows()

In [ ]:
#평균이동필터
"""
영상의 일정한 반경 크기의 커널 윈도로 픽셀 값의 평균 값을 커널의 중심으로 바꿔서
이동하는 것을 반복하다 보면 그 주변에서 가장 밀집한 곳을 찾을 수 있음.
이런 방법으로 특정 공간 내의 분포의 피크를 찾는 방법을 평균이동이라고함.
이동을 시작한 지점에서 중지한 지점까지를 하나로 묶으면 연결된 영역을 찾을 수 있음. 
같은 방법으로 가장 빈도가 많은 색상을 구해서 연결된 영역의 모든 픽셀 값으로 바꾸면 
연결된 영역을 구분할 수 있음. 이 결과를 영상으로 보면 마치 사람이 포스터를 그린 것과 비슷한 효과

cv2.pyrMeanShiftFiltering(src, sp, sr[, dst, maxLevel, termcrit])
  -src:입력영상
  -sp:공간 윈도 반지름 크기
  -sr:색상윈도 반지름 크기
  -maxLevel:이미지 피라미드 최대 레벨 #이미지 피라미드는 확대 축소한거 의미
  -termcrit:반복 중지 요건
    -type=cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS:중지 형식
      -cv2.TERM_CRITERIA_EPS:정확도가 최소 정확도보다 작아지면 중지
      -cv2.TERM_CRITERIA_MAX_ITER:최대 반복횟수(max_iter)에 도달하면 중지
      -cv2.TERM_CRITERIA_COUNT:cv2.TERM_CRITERIA_MAX_ITER와 동일
    -max_iter:최대 반복 횟수
    -epsilon:최소 정확도
"""

In [26]:
#평균 이동 세그멘테이션 필터
import cv2
import numpy as np

img = cv2.imread('./img/taekwonv1.jpg')
#트랙바 이벤트 처리 함수
def onChange(x):
    #sp, sr, level 선택 값 수집
    sp= cv2.getTrackbarPos('sp', 'img')
    sr= cv2.getTrackbarPos('sr', 'img')
    lv= cv2.getTrackbarPos('lv', 'img')
    
    #평균이동필터적용
    mean = cv2.pyrMeanShiftFiltering(img, sp, sr, None, lv)
    #변환 이미지 출력
    cv2.imshow('img', np.hstack((img, mean)))
    
#초기화면 출력
cv2.imshow('img', np.hstack((img,img)))
#트랙바 이벤트 함수 연결
cv2.createTrackbar('sp', 'img', 0, 100, onChange)
cv2.createTrackbar('sr', 'img', 0, 100, onChange)
cv2.createTrackbar('lv', 'img', 0, 5, onChange)
cv2.waitKey()
cv2.destroyAllWindows()

In [41]:
#실전 워크숍
#5개의 도형이 들어있는 영상에서 각각 도형의 이름을 알아맞히는 프로그램을 만들기
#도형 알아맞히기 워크숍
import cv2
import numpy as np

#이미지를 읽어 들여서 그레이 스케일 및 스레시홀드 변환
img = cv2.imread('./img/5shapes.jpg')
img2 = img.copy()
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

#컨투어 찾기
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    #각 컨투어에 근사 컨투어로 단순화
    approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
    #꼭짓점의 개수
    vertices = len(approx)
    print('verticies:', vertices)
    
    #중심점 찾기
    mmt = cv2.moments(contour)
    cx, cy = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])
    
    name='Unknown'
    if vertices == 3: #꼭짓점 3개는 삼각형
        name='Triangle'
        color=(0,255,0)
    elif vertices == 4: #꼭짓점 4개는 사각형
        x,y,w,h = cv2.boundingRect(contour)
        if abs(w-h) <=3: #폭과 높이의 차가 3보다 작으면 정사각형
            name='Square'
            color = (0, 215, 255)
        else:    #폭과 높이의 차가 3보다 크면 직사각형
            name = 'Rectangle'
            color = (0,0,255)
    elif vertices == 10: #꼭짓점 개수 10개는 별
        name='star'
        color=(255,255,0)
    elif vertices >= 15: #꼭짓점 10개 이상이면 원
        name='Circle'
        color=(0,255,255)
    #컨투어 그리기
    cv2.drawContours(img2, [contour], -1, color, -1)
    #도형 이름 출력
    cv2.putText(img2, name, (cx-50, cy), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (100,100,100), 1)

cv2.imshow('Input Shapes', img)
cv2.imshow('Recognizing Shapes', img2)
cv2.waitKey()
cv2.destroyAllWindows()
    

verticies: 10
verticies: 16
verticies: 3
verticies: 4
verticies: 4


In [1]:
#문서 스캐너


import cv2
import numpy as np

win_name = 'scan'
#이미지 읽기
img = cv2.imread('./img/paper.jpg')
cv2.imshow('original', img)
cv2.waitKey()
draw = img.copy()

#그레이 스케일 변환 및 캐니 엣지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0) #가우시안 블러로 노이즈 제거
edged = cv2.Canny(gray, 75, 200) #캐니엣지로 경계 검출
cv2.imshow(win_name, edged)
cv2.waitKey()

#컨투어 찾기
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#모든 컨투어 그리기
cv2.drawContours(draw, cnts, -1, (0,255,0))
cv2.imshow(win_name, draw)
cv2.waitKey(0)

#컨투어들 중에 영역 크기순으로 정렬
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    #영역이 가장 큰 컨투어부터 근사 컨투어 단순화
    peri = cv2.arcLength(c, True) #둘레길이
    #둘레 길이의 0.02 근사값으로 근사화
    vertices = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(vertices) ==4:#근사한 꼭짓점이 4개면 중지
        break
pts=vertices.reshape(4,2) #N x 1 X 2 배열을 4 x 2 크기로 조정
for x,y in pts:
    cv2.circle(draw, (x,y), 10, (0,255,0), -1) #좌표에 초록색 동그라미 표시
    cv2.imshow(win_name, draw)
    cv2.waitKey()
    merged = np.hstack((img, draw))

In [3]:
import cv2
import numpy as np

win_name = 'scan'
# 이미지 읽기
img = cv2.imread("./img/paper.jpg")
cv2.imshow('original', img)
cv2.waitKey(0)
draw = img.copy()

# 그레이스 스케일 변환 및 케니 엣지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 가우시안 블러로 노이즈 제거
edged = cv2.Canny(gray, 75, 200)    # 케니 엣지로 경계 검출
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# 컨투어 찾기
(_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, \
                                                cv2.CHAIN_APPROX_SIMPLE)
# 모든 컨투어 그리기
cv2.drawContours(draw, cnts, -1, (0,255,0))
cv2.imshow(win_name, draw)
cv2.waitKey(0)

# 컨투어들 중에 영역 크기 순으로 정렬
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    # 영역이 가장 큰 컨투어 부터 근사 컨투어 단순화
    peri = cv2.arcLength(c, True)   # 둘레 길이
    # 둘레 길이의 0.02 근사값으로 근사화
    vertices = cv2.approxPolyDP(c, 0.02 * peri, True) 
    if len(vertices) == 4: # 근사한 꼭지점이 4개면 중지
        break
pts = vertices.reshape(4, 2) # N x 1 x 2 배열을 4 x 2크기로 조정
for x,y in pts:
    cv2.circle(draw, (x,y), 10, (0,255,0), -1) # 좌표에 초록색 동그라미 표시
cv2.imshow(win_name, draw)
cv2.waitKey(0)
merged = np.hstack((img, draw))



ValueError: not enough values to unpack (expected 3, got 2)

In [2]:
import cv2
import numpy as np

win_name = 'scan'
# 이미지 읽기
img = cv2.imread("./img/paper.jpg")
cv2.imshow('original', img)
cv2.waitKey(0)
draw = img.copy()

# 그레이스 스케일 변환 및 케니 엣지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 가우시안 블러로 노이즈 제거
edged = cv2.Canny(gray, 75, 200)    # 케니 엣지로 경계 검출
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# 컨투어 찾기
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, \
                                                cv2.CHAIN_APPROX_SIMPLE)
# 모든 컨투어 그리기
cv2.drawContours(draw, cnts, -1, (0,255,0))
cv2.imshow(win_name, draw)
cv2.waitKey(0)

# 컨투어들 중에 영역 크기 순으로 정렬
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    # 영역이 가장 큰 컨투어 부터 근사 컨투어 단순화
    peri = cv2.arcLength(c, True)   # 둘레 길이
    # 둘레 길이의 0.02 근사값으로 근사화
    vertices = cv2.approxPolyDP(c, 0.02 * peri, True) 
    if len(vertices) == 4: # 근사한 꼭지점이 4개면 중지
        break
pts = vertices.reshape(4, 2) # N x 1 x 2 배열을 4 x 2크기로 조정
for x,y in pts:
    cv2.circle(draw, (x,y), 10, (0,255,0), -1) # 좌표에 초록색 동그라미 표시
cv2.imshow(win_name, draw)
cv2.waitKey(0)
merged = np.hstack((img, draw))

#맨마지막이 안됨

In [1]:
#위와 같은 결과
#좌표 4개 중 상하좌우 찾기
sm = pts.sum(axis=1) #4쌍의 좌표 각각 x+y 계산
diff = np.diff(pts, axis=1) #4쌍의 좌표 각각 x-y계산

topLeft = pts[np.argmin(sm)] #x+y가 가장 작은 값이 좌상단 좌표
bottomRight = pts[np.argmax(sm)] #x+y가 가장 큰 값이 좌상단 좌표
topRight = pts[np.argmin(diff)] #x-y가 가장 작은 값이 우상단 좌표
bottomLeft = pts[np.argmax(diff)] #x-y가 가장 큰 값이 좌하단 좌표

#변환 전 4개 좌표
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

#변환 후 영상에 사용할 서류의 폭과 높이 계산
w1= abs(bottomRight[0] - bottomLeft[0])# 상단 좌우 좌표 간의 거리
w2 = abs(topRight[0] - topLeft[0]) #하단 좌우 좌표 간의 거리
h1 = abs(topRight[1] - bottomRight[1])#우측 상하 좌표 간의 거리
h2 = abs(topLeft[1] - bottomLeft[1]) #좌측 상하 좌표 간의 거리
width = max([w1, w2]) #두 좌우 거리 간의 최대 값이 서류의 폭
height = max([h1, h2]) #두 상하 거리 간의 최대 값이 서류의 높이

#변환 후 4개의 좌표
pts2 = np.float32([[0,0], [width-1,0], [width-1, height-1], [0,height-1]])

#변환행렬 계산
mtrx = cv2.getPerspectiveTransform(pts1, pts2)
#원근변환 적용
result = cv2.warpPerspective(img, mtrx, (width, height))
cv2.imshow(win_name, result)
cv2.waitKey()
cv2.destroyAllWindows()

NameError: name 'pts' is not defined

In [2]:
import cv2
import numpy as np

win_name = 'scan'
# 이미지 읽기
img = cv2.imread("./img/paper.jpg")
cv2.imshow('original', img)
cv2.waitKey(0)
draw = img.copy()

# 그레이스 스케일 변환 및 케니 엣지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 가우시안 블러로 노이즈 제거
edged = cv2.Canny(gray, 75, 200)    # 케니 엣지로 경계 검출
cv2.imshow(win_name, edged)
cv2.waitKey(0)

# 컨투어 찾기
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, \
                                                cv2.CHAIN_APPROX_SIMPLE)
# 모든 컨투어 그리기
cv2.drawContours(draw, cnts, -1, (0,255,0))
cv2.imshow(win_name, draw)
cv2.waitKey(0)

# 컨투어들 중에 영역 크기 순으로 정렬
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    # 영역이 가장 큰 컨투어 부터 근사 컨투어 단순화
    peri = cv2.arcLength(c, True)   # 둘레 길이
    # 둘레 길이의 0.02 근사값으로 근사화
    vertices = cv2.approxPolyDP(c, 0.02 * peri, True) 
    if len(vertices) == 4: # 근사한 꼭지점이 4개면 중지
        break
pts = vertices.reshape(4, 2) # N x 1 x 2 배열을 4 x 2크기로 조정
for x,y in pts:
    cv2.circle(draw, (x,y), 10, (0,255,0), -1) # 좌표에 초록색 동그라미 표시
cv2.imshow(win_name, draw)
cv2.waitKey(0)
merged = np.hstack((img, draw))

#맨마지막이 안됨
#위와 같은 결과
#좌표 4개 중 상하좌우 찾기
sm = pts.sum(axis=1) #4쌍의 좌표 각각 x+y 계산
diff = np.diff(pts, axis=1) #4쌍의 좌표 각각 x-y계산

topLeft = pts[np.argmin(sm)] #x+y가 가장 작은 값이 좌상단 좌표
bottomRight = pts[np.argmax(sm)] #x+y가 가장 큰 값이 좌상단 좌표
topRight = pts[np.argmin(diff)] #x-y가 가장 작은 값이 우상단 좌표
bottomLeft = pts[np.argmax(diff)] #x-y가 가장 큰 값이 좌하단 좌표

#변환 전 4개 좌표
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])

#변환 후 영상에 사용할 서류의 폭과 높이 계산
w1= abs(bottomRight[0] - bottomLeft[0])# 상단 좌우 좌표 간의 거리
w2 = abs(topRight[0] - topLeft[0]) #하단 좌우 좌표 간의 거리
h1 = abs(topRight[1] - bottomRight[1])#우측 상하 좌표 간의 거리
h2 = abs(topLeft[1] - bottomLeft[1]) #좌측 상하 좌표 간의 거리
width = max([w1, w2]) #두 좌우 거리 간의 최대 값이 서류의 폭
height = max([h1, h2]) #두 상하 거리 간의 최대 값이 서류의 높이

#변환 후 4개의 좌표
pts2 = np.float32([[0,0], [width-1,0], [width-1, height-1], [0,height-1]])

#변환행렬 계산
mtrx = cv2.getPerspectiveTransform(pts1, pts2)
#원근변환 적용
result = cv2.warpPerspective(img, mtrx, (width, height))
cv2.imshow(win_name, result)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
#동전개수 세기 워크숍 풀이
import cv2
import numpy as np

#이미지 읽기
img = cv2.imread('./img/coins_connected.jpg')
rows, cols = img.shape[:2]
cv2.imshow('original', img)

#동전 표면을 흐릿하게 피라미드 평균 시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)

#바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)

_,thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
#거리변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
#거리값을 0-255로 변환
dst = (dst/(dst.max() - dst.min()) *255).astype(np.uint8)
cv2.imshow('dst', dst)

#거리변환 결과에서 로컬 최대 값 구하기
#팽창 적용(동전 크기정도의 구조화 요소 필요)
localMx = cv2.dilate(dst, np.ones((50,50), np.uint8))
#로컬 최대값을 저장할 배열 생성
lm = np.zeros((rows, cols), np.uint8)
#팽창 적용전 이미지와 같은 픽셀이 로컬 최대 값이므로 255로 설정
lm[(localMx==dst) & (dst !=0)] =255
cv2.imshow('localMx', lm)

#로컬 최대값으로 색 채우기
##로컬 최대 값이 있는 좌표 구하기
seeds = np.where(lm=255)
seed= np.stack((seeds[1], seeds[0]), axis=-1)
##색 채우기를 위한 채우기 마스크 생성
fill_mask = np.zeros((rows+2, cols+2), np.uint8)
for x, y in seed:
    ##로컬 최대값을 시드로 해서 평균 시프트 영상에 색채우기
    ret = cv2.floodFill(mean, fill_mask, (x,y), (255,255,255), (10,10,10), (10,10,10))
    cv2.imshow('floodFill', mean)
    
    #색 채우기 적용한 영상에 다시 거리 변환 적용
    gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
    dst = (dst/(dst/(dst.max()-dst.min())) *255).astype(np.uint8)
    cv2.imshow('dst2', dst)
    
    #거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
    ret, sure_fg = cv2.threshold(dst, 0.5*dst.max(), 255,0)
    cv2.imshow('sure_fg', sure_fg)
    
    #거리변환 결과를 반전해서 확실한 배결 찾기
    _, bh_th = cv2.threshold(dst, 0.3*dst.max(), 255, cv2.THRESH_BINARY_INV)
    bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2,5)
    bg_dst = ((bg_dst/(bg_dst.max() - bg_dst.min()))*255).astype(np.uint8)
    ret, sure_bg = cv2.threshold(bg_dst, 0.3*bg_dst.max(), 255, cv2.THRESH_BINARY)
    cv2.imshow('sure_bg', sure_bg)
    
    #불확실한 영역 설정:확실한 배경을 반전해서 확실한 전경을 빽;
    ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255, cv2.THRESH_BINARY_INV)
    unknown = cv2.substract(inv_sure_bg, sure_fg)
    cv2.imshow('unknown', unknown)
    
    #연결된 요소 레이블링
    _, markers = cv2.connectedComponents(sure_fg)
    
    #레이블링을 1씩 증가시키고 0번 레이블 알 수 없는 영역을 0번 레이블로 설정
    markers=marker+1
    markers[unknown ==255] =0
    print('워터쉐드 전:', np.unique(markers))
    colors =[]
    marker_show = np.zeros_like(img)
    for mid in np.unique(markers):#선택한 마커 아이디 갯수 만큼 반복
        color = [int(j) for j in np.random.randint(0,255,3)]
        colors.append((mid, color))
        marker_show[markers=mid]=color
        coords = np.where(markers=mid)
        x,y = coords[1][0], coords[0][0]
        cv2.PutText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PALIN, 2, (255,255,255))
cv2.imshow('before',marker_show)

#레이블링이 완성된 마커로 워터 쉐드 적용
markers = cv2.watershed(img, markers)
print('워터쉐트 후:', np.unique(markers))

for mid, color in colors: #선택한 마커 아이디 갯수 만큼 반복
    marker_show[markers=mid]=color
    coords = np.where(markers==mid)
    if coords[0].size <=0:
        continue
    x,y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
    
    marker_show[markers==1] = (0,255,0)
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))

marker_show[markers=-1] = (0,255,0)
cv2.imshow('watershed marker', marker_show)

img[markers==-1] = (0,255,0)
cv2.imshow('watershed', img)

#동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
#배경 마스크 생성
mask[markers!=1]=255
#배경 지우기
nobg = cv2.bitwise_and(img, img, mask=mask)
#동전만 있는 라벨 생성(배경(1), 경계(-1) 없는)
coin_label = [l for l in np.unique(markers) if (l !=1 and !=1)]
#동전 라벨 순회 하면서 동전 영역만 추출
for i, label in enumerate(coin_label):
    mask-[:,:] = 0
    #해당 동전 추출 마스크 생성
    mask[markers==label] =255
    #동전 영역만 마스크로 추출
    coins = cv2.bitwise_and(img, img, mask=mask)
    #동전 하나만 있는 곳에서 최외곽 컨투어 추출
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    #동전을 감싸는 사각형 좌표
    x,y,w,h = cv2.boundingRect(contour[0])
    #동전 영역만 추출해서 출력
    coin = coins[y:y+h, x:x+w]
    cv2.imshow('coin%d'%(i+1), coin)
    cv2.imwrite('./img/coin_test/coin%d.jpg'%(i+1), coin)
cv2.waitKey()
cv2.destroyAllWindows()

SyntaxError: invalid syntax (Temp/ipykernel_3544/362322644.py, line 83)

In [2]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread('./img/coins_connected.jpg')
rows, cols = img.shape[:2]
cv2.imshow('original', img)


# 동전 표면을 흐릿하게 피라미드평균시프트 적용
mean = cv2.pyrMeanShiftFiltering(img, 20, 50)
cv2.imshow('mean', mean)
# 바이너리 이미지 변환
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3,3), 0)

_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imshow('thresh', thresh)
# 거리 변환
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 3)
# 거리 값을 0 ~255로 변환
dst = ( dst / (dst.max() - dst.min()) * 255 ).astype(np.uint8)
cv2.imshow('dst', dst)

# 거리 변환결과에서 로칼 최대 값 구하기
## 팽창 적용(동전 크기 정도의 구조화 요소 필요),
localMx = cv2.dilate(dst, np.ones((50,50), np.uint8))
## 로칼 최대 값 저장 할 배열 생성
lm = np.zeros((rows, cols), np.uint8)
## 팽창 적용전 이미지와 같은 픽셀이 로컬 최대 값이므로 255로 설정
lm[(localMx==dst) & (dst != 0)] = 255
cv2.imshow('localMx', lm)

# 로컬 최대값으로 색 채우기
## 로컬 최대 값이 있는 좌표 구하기
seeds = np.where(lm ==255)
seed = np.stack( (seeds[1], seeds[0]), axis=-1)
## 색 채우기를 위한 채우기 마스크 생성
fill_mask = np.zeros((rows+2, cols+2), np.uint8)
for x,y in seed:
    ## 로칼 최대값을 시드로해서 평균 시프트 영상에 색채우기 
    ret = cv2.floodFill(mean, fill_mask, (x,y), (255,255,255), \
                                            (10,10,10), (10,10,10))
cv2.imshow('floodFill', mean)

# 색 채우기 적용한 영상에 다시 거리 변환 적용
gray = cv2.cvtColor(mean, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5,5), 0)

ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
dst = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
dst = ( (dst / (dst.max() - dst.min())) * 255 ).astype(np.uint8)
cv2.imshow('dst2', dst)

# 거리 변환 결과값의 절반 이상을 차지한 영역은 확실한 전경으로 설정
ret, sure_fg = cv2.threshold(dst, 0.5*dst.max(), 255,0)
cv2.imshow('sure_fg', sure_fg)

# 거리 변환 결과를 반전해서 확실한 배경 찾기
_, bg_th = cv2.threshold(dst, 0.3*dst.max(),  255, cv2.THRESH_BINARY_INV)
bg_dst = cv2.distanceTransform(bg_th, cv2.DIST_L2, 5)
bg_dst = ( (bg_dst / (bg_dst.max() - bg_dst.min())) * 255 ).astype(np.uint8)
ret, sure_bg = cv2.threshold(bg_dst, 0.3*bg_dst.max(), 255,cv2.THRESH_BINARY)
cv2.imshow('sure_bg', sure_bg)


# 불확실한 영역 설정 : 확실한 배경을 반전해서 확실한 전경을 빼기
ret, inv_sure_bg = cv2.threshold(sure_bg, 127, 255,cv2.THRESH_BINARY_INV)
unkown = cv2.subtract(inv_sure_bg, sure_fg)
cv2.imshow('unkown', unkown)

# 연결된 요소 레이블링
_, markers = cv2.connectedComponents(sure_fg)

# 레이블링을 1씩 증가 시키고 0번 레이블 알 수 없는 영역을 0번 레이블로 설정
markers = markers+1
markers[unkown ==255] = 0
print("워터쉐드 전:", np.unique(markers))
colors = []
marker_show = np.zeros_like(img)
for mid in np.unique(markers): # 선택한 마커 아이디 갯수 만큼 반복
    color = [int(j) for j in np.random.randint(0,255, 3)]
    colors.append((mid, color))
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, \
                                                             2, (255,255,255))
cv2.imshow('before', marker_show)

# 레이블링이 완성된 마커로 워터 쉐드 적용
markers = cv2.watershed(img, markers)
print("워터쉐드 후:", np.unique(markers))

for mid, color in colors: # 선택한 마커 아이디 갯수 만큼 반복
    marker_show[markers==mid] = color
    coords = np.where(markers==mid)
    if coords[0].size <= 0 : 
        continue
    x, y = coords[1][0], coords[0][0]
    cv2.putText(marker_show, str(mid), (x+20, y+20), cv2.FONT_HERSHEY_PLAIN, \
                                                             2, (255,255,255))
marker_show[markers==-1] = (0,255,0)
cv2.imshow('watershed marker', marker_show)

img[markers==-1] = (0,255,0)
cv2.imshow('watershed', img)

# 동전 추출을 위한 마스킹 생성
mask = np.zeros((rows, cols), np.uint8)
# 배경 마스크 생성
mask[markers!=1] = 255
# 배경 지우기
nobg = cv2.bitwise_and(img, img, mask=mask)
# 동전만 있는 라벨 생성 (배경(1), 경계(-1) 없는)
coin_label = [l for l in np.unique(markers) if (l != 1 and l !=-1)]
# 동전 라벨 순회 하면서 동전 영역만 추출
for i, label in enumerate(coin_label):
    mask[:,:] = 0
    # 해당 동전 추출 마스크 생성
    mask[markers ==label] = 255
    # 동전 영역만 마스크로 추출
    coins = cv2.bitwise_and(img, img, mask=mask)
    # 동전 하나만 있는 곳에서 최외곽 컨투어 추출
    _, contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,\
                                         cv2.CHAIN_APPROX_NONE)
    # 동전을 감싸는 사각형 좌표
    x,y,w,h = cv2.boundingRect(contour[0])
    # 동전 영역만 추출해서 출력
    coin = coins[y:y+h, x:x+w]
    cv2.imshow('coin%d'%(i+1), coin)
    cv2.imwrite('../img/coin_test/coin%d.jpg'%(i+1), coin)
cv2.waitKey()
cv2.destroyAllWindows()

워터쉐드 전: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
워터쉐드 후: [-1  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


ValueError: not enough values to unpack (expected 3, got 2)